# 1. 데이터 가져오기

In [1]:
import torchvision.transforms as T
import torchvision
import torch
from torch.utils.data import DataLoader

download_root = './MNIST_DATASET'

mnist_transform = T.Compose([
    T.ToTensor(),
])

train_dataset = torchvision.datasets.MNIST(download_root, transform=mnist_transform, train=True, download=False)
test_dataset = torchvision.datasets.MNIST(download_root, transform=mnist_transform, train=False, download=False)

total_size = len(train_dataset)
train_num, valid_num = int(total_size * 0.8), int(total_size * 0.2)
train_dataset,valid_dataset = torch.utils.data.random_split(train_dataset, [train_num, valid_num])

batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
valid_dataloader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = False)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [2]:
for data, label in train_dataloader:
    print(data.shape)
    break

torch.Size([32, 1, 28, 28])


# 2.모델 만들기

In [3]:
from pytorch_lightning import LightningModule, Trainer
import torch.optim as optim
import torchmetrics
import torch.nn as nn

from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger

import wandb

In [4]:
class RNNClassifire(LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, lr):
        super().__init__()

        self.learning_rate = lr
        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes = num_classes)

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes

        self.rnn = nn.RNN(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first = True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        '''
        INPUT :
            x : [batch_size, sequence_length, input_size] => [batch_size, width, height]
        OUTPUT :
            out : [batch_size, num_classes]
        '''
        x = x.squeeze()
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr = self.learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        loss = self.criterion(y_hat, y)

        _, preds = torch.max(y_hat, dim = 1)
        acc = self.accuracy(preds, y)

        self.log(f"valid_loss", loss, on_step = False, on_epoch = True, logger = True)
        self.log(f"valid_acc", acc, on_step = False, on_epoch = True, logger = True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        loss = self.criterion(y_hat, y)

        _, preds = torch.max(y_hat, dim = 1)
        acc = self.accuracy(preds, y)

        self.log(f"valid_loss", loss, on_step = False, on_epoch = True, logger = True)
        self.log(f"valid_acc", acc, on_step = False, on_epoch = True, logger = True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        loss = self.criterion(y_hat, y)

        _, preds = torch.max(y_hat, dim = 1)
        acc = self.accuracy(preds, y)

        self.log(f"test_loss", loss, on_step = False, on_epoch = True, logger = True)
        self.log(f"test_acc", acc, on_step = False, on_epoch = True, logger = True)

    def predict_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        _, preds = torch.max(y_hat, dim = 1)

        return preds


# 3. 모델 실행 및 평가

In [5]:
model = RNNClassifire(input_size = 28, hidden_size = 128, num_layers = 2, num_classes = 10, lr = 0.001)

early_stopping = EarlyStopping(monitor = 'valid_loss', mode = 'min', patience=5)
lr_mointor = LearningRateMonitor(logging_interval = 'epoch')

wandb_logger = WandbLogger(project = 'MNIST_RNN')

trainer = Trainer(
    max_epochs = 50,
    accelerator = 'auto',
    callbacks = [early_stopping, lr_mointor],
    logger = wandb_logger
)

trainer.fit(
    model,
    train_dataloader,
    valid_dataloader
)

trainer.test(model, test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tjsgh2770 (prefer_leee). Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | criterion | CrossEntropyLoss   | 0      | train
1 | accuracy  | MulticlassAccuracy | 0      | train
2 | rnn       | RNN                | 53.2 K | train
3 | fc        | Linear             | 1.3 K  | train
---------------------------------------------------------
54.5 K    Trainable params
0         Non-trainable params
54.5 K    Total params
0.218     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\tjsgh\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
c:\Users\tjsgh\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\tjsgh\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9833999872207642     │
│         test_loss         │    0.07461227476596832    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.07461227476596832, 'test_acc': 0.9833999872207642}]

: 